In [2]:
import pandas as pd

In [37]:
data = pd.read_csv("../../data/clean-data/reddit-posts.csv")

In [38]:
data.drop(['Unnamed: 0', 'full_text'], axis = 1, inplace = True)

In [39]:
data.sample(5)

,title,selftext,author,date,score,subreddit,tickers_mentioned,clean_text
116,See you in Premarket NVDA bears,jobs not finished.,lilmutt32,2025-05-28 21:27:33,2821,wallstreetbets,['NVDA'],see you in premarket nvda bears jobs not finished
3,Investing is simple! It’s not hard at all!,Investing is simple as pie. I’ve only been inv...,general-ring2780,2025-04-28 01:31:28,0,investing,['NVDA'],investing is simple its not hard at all invest...
141,Wendys or McDonalds,Day 3 or something of posting my googl calls d...,microsofttothemoon,2025-06-03 23:24:01,55,wallstreetbets,['GOOGL'],wendys or mcdonalds day 3 or something of post...
56,Holding NVDA over weekend,To be honest with the Moody news it may look b...,hot-lengthiness1251,2025-05-16 22:05:05,30,wallstreetbets,['NVDA'],holding nvda over weekend to be honest with th...
31,"Over the past 10 years, annual CEO pay increas...",That stat is a finding from research firm Equi...,barronsmag,2025-05-13 15:49:29,97,stocks,"['AAPL', 'MSFT']",over the past 10 years annual ceo pay increase...


In [40]:
data.isna().sum()

title                 0
selftext             14
author                0
date                  0
score                 0
subreddit             0
tickers_mentioned     0
clean_text            0
dtype: int64

In [41]:
data.duplicated().sum()

np.int64(0)

In [42]:
data['date'] = pd.to_datetime(data['date'])
data['date'] = data['date'].dt.date

In [43]:
data[data['score'] > 5].sample(5)

,title,selftext,author,date,score,subreddit,tickers_mentioned,clean_text
64,Lil baby gains $NVDA,"Cashed out a little bit last week, but holding...",deep_inda_money,2025-05-19,92,wallstreetbets,['NVDA'],lil baby gains $nvda cashed out a little bit l...
94,Nvda (Nvidia) upcoming earnings will either pu...,"The last few nvidia earnings have been okay, a...",daydreamer1015,2025-05-25,101,stocks,['NVDA'],nvda nvidia upcoming earnings will either pump...
47,These $132C NVDA shares gave me a head start 😅...,NaN,kujothecat,2025-05-15,17,wallstreetbets,['NVDA'],these $132c nvda shares gave me a head start d...
124,NVIDIA's AI dominance is solid! Q1 data center...,After the U.S. stock market session on Wednesd...,wilbo75771,2025-05-29,69,wallstreetbets,['NVDA'],nvidias ai dominance is solid q1 data center b...
115,NVDA earnings,Again we see that they are unstoppable\n\n* Ad...,swisscowonmoon,2025-05-28,291,stocks,['NVDA'],nvda earnings again we see that they are unsto...


In [44]:
data['tickers_mentioned'] = data['tickers_mentioned'].fillna('').str.replace(' ', '')
data = data[data['tickers_mentioned'] != '']  # Drop posts with no tickers

In [45]:
!pip install nltk

In [46]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/aryagupta/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [47]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Apply with explicit text argument
data['sentiment_scores'] = data['clean_text'].apply(lambda text: sia.polarity_scores(str(text)))

In [48]:
data['compound'] = data['sentiment_scores'].apply(lambda x: x['compound'])

In [49]:
def classify_sentiment(compound):
    if compound >= 0.05:
        return 'positive'
    elif compound <= -0.05:
        return 'negative'
    else:
        return 'neutral'

data['sentiment'] = data['compound'].apply(classify_sentiment)

In [51]:
data.drop(['sentiment_scores', 'selftext', 'title'], axis = 1, inplace = True)

In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   author             161 non-null    object 
 1   date               161 non-null    object 
 2   score              161 non-null    int64  
 3   subreddit          161 non-null    object 
 4   tickers_mentioned  161 non-null    object 
 5   clean_text         161 non-null    object 
 6   compound           161 non-null    float64
 7   sentiment          161 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 10.2+ KB


In [60]:
import ast

def stringify_tickers(x):
    if isinstance(x, list):
        return ','.join(x)
    try:
        # Try to parse string representation of list
        parsed = ast.literal_eval(x)
        if isinstance(parsed, list):
            return ','.join(parsed)
        else:
            return str(x)
    except:
        return str(x)

data['tickers_mentioned'] = data['tickers_mentioned'].apply(stringify_tickers)

In [62]:
data.sample(5)

,author,date,score,subreddit,tickers_mentioned,clean_text,compound,sentiment
129,hot-lengthiness1251,2025-05-31,147,wallstreetbets,NVDA,yolo nvda calls could be the best decision of ...,0.6124,positive
91,thelankyasian,2025-05-24,651,wallstreetbets,NVDA,i recently discovered nvda options are a hell ...,-0.5423,negative
75,callsonreddit,2025-05-21,66,wallstreetbets,NVDA,us to keep china chip curbs spurning nvidias c...,0.9432,positive
100,wilbo75771,2025-05-26,258,wallstreetbets,"GOOGL,AAPL,MSFT,NVDA,AMZN",nvidia artificial intelligence nvidia nvda has...,0.9883,positive
150,chouprojects,2025-06-05,195,wallstreetbets,NVDA,cloudflare $net the next $pltr alright losers ...,0.9615,positive


In [63]:
data.to_csv('../../data/clean-data/cleaned_reddit_v2.csv')